In [21]:
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from shutil import copyfile
from tensorflow.keras.callbacks import Callback
from ipywidgets import interact, widgets
import time
from tqdm import tqdm

In [44]:
def fit_rotated_ellipse_ransac(data,iter=50,sample_num=10,offset=80.0):

    count_max = 0
    effective_sample = None

    for i in range(iter):
        sample = np.random.choice(len(data), sample_num, replace=False)

        xs = data[sample][:,0].reshape(-1,1)
        ys = data[sample][:,1].reshape(-1,1)

        J = np.mat( np.hstack((xs*ys,ys**2,xs, ys, np.ones_like(xs,dtype=np.float64))) )
        Y = np.mat(-1*xs**2)
        P= (J.T * J).I * J.T * Y

        # fitter a*x**2 + b*x*y + c*y**2 + d*x + e*y + f = 0
        a = 1.0; b= P[0,0]; c= P[1,0]; d = P[2,0]; e= P[3,0]; f=P[4,0];
        ellipse_model = lambda x,y : a*x**2 + b*x*y + c*y**2 + d*x + e*y + f

        # threshold 
        ran_sample = np.array([[x,y] for (x,y) in data if np.abs(ellipse_model(x,y)) < offset ])

        if(len(ran_sample) > count_max):
            count_max = len(ran_sample) 
            effective_sample = ran_sample

    return fit_rotated_ellipse(effective_sample)


def fit_rotated_ellipse(data):

    xs = data[:,0].reshape(-1,1) 
    ys = data[:,1].reshape(-1,1)

    J = np.mat( np.hstack((xs*ys,ys**2,xs, ys, np.ones_like(xs,dtype=np.float64))) )
    Y = np.mat(-1*xs**2)
    P= (J.T * J).I * J.T * Y

    a = 1.0; b= P[0,0]; c= P[1,0]; d = P[2,0]; e= P[3,0]; f=P[4,0];
    theta = 0.5* np.arctan(b/(a-c))  
    
    cx = (2*c*d - b*e)/(b**2-4*a*c)
    cy = (2*a*e - b*d)/(b**2-4*a*c)

    cu = a*cx**2 + b*cx*cy + c*cy**2 -f
    w= np.sqrt(cu/(a*np.cos(theta)**2 + b* np.cos(theta)*np.sin(theta) + c*np.sin(theta)**2))
    h= np.sqrt(cu/(a*np.sin(theta)**2 - b* np.cos(theta)*np.sin(theta) + c*np.cos(theta)**2))

    ellipse_model = lambda x,y : a*x**2 + b*x*y + c*y**2 + d*x + e*y + f

    error_sum = np.sum([ellipse_model(x,y) for x,y in data])
    #print('fitting error = %.3f' % (error_sum))

    return (cx,cy,w,h,theta)

def apply_blur(image, blur_amount=5):
    return cv2.GaussianBlur(image, (blur_amount, blur_amount), 0)

In [103]:
def fitPupil(image,circ_thresh=0.5,thresh_val=60,kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)),min_thresh=170,max_thresh=280,resize=1):
        #print(image)
        
        temp_image = image.copy()
        

        inf_img = temp_image.copy()
        
        image_gray = cv2.cvtColor(temp_image , cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(image_gray,(3,3),0)
        
        ret,thresh1 = cv2.threshold(blur,thresh_val,255,cv2.THRESH_BINARY)
        
        opening = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)
        
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        
        
        #size_el = 0
        temp_image  = 255 - closing
        
        
        contours, hierarchy = cv2.findContours(temp_image , cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        hull = []
        for i in range(len(contours)):
          hull.append(cv2.convexHull(contours[i], False)) 
        
        cx,cy,w,h,theta = 0.0,0.0,0.0,0.0,0.0
        for con in hull:
            
            approx = cv2.approxPolyDP(con, 0.01 * cv2.arcLength(con,True),True)
            area = cv2.contourArea(con)
            perimeter = cv2.arcLength(con, True)
            circularity = 4 * np.pi * (area / (perimeter * perimeter))


            if circularity > circ_thresh:
                
            
        
        
                if(len(approx) > 10 and area > 400):
                        try:
                        
                            cx,cy,w,h,theta = fit_rotated_ellipse_ransac(con.reshape(-1,2))
                            #size_el = ellipse_circumference(w,h)

                            # if size_el > min_thresh and size_el < max_thresh:
                            
                            #xcoordinates.append(cx)
                            #ycoordinates.append(cy)
                            cv2.ellipse(inf_img,(int(cx),int(cy)),(int(w),int(h)),theta*180.0/np.pi,0.0,360.0,(0,255,255),1)
                            inf_img = cv2.drawMarker(inf_img, (int(cx),int(cy)),(0, 255, 255),cv2.MARKER_CROSS,2,1)
                            #else: cx,cy,w,h,theta = 0.0,0.0,0.0,0.0,0.0
                            
                        except Exception as e: pass
            
        return inf_img,[cx,cy,w,h,theta]
    

In [24]:
def draw_text(frame, text_lines, position):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_thickness = 1
    color = (255, 255, 255)  # White color in BGR
    
    for i, line in enumerate(text_lines):
        y_offset = i * 20  # Adjust vertical offset
        cv2.putText(frame, line, (position[0], position[1] + y_offset), font, font_scale, color, font_thickness, cv2.LINE_AA)



In [25]:
# Function to calculate distance between two points
def distance(point1, point2):
    return np.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)


In [26]:

def calculate_velocity(object_data, prev_object_data, frame_rate):
  """
  This function calculates the velocity of an object in pixels per frame.

  Args:
      object_data: A dictionary containing the current object data (x, y, width, height, angle).
      prev_object_data: A dictionary containing the previous object data (from the previous frame).
      frame_rate: The frame rate of the video.

  Returns:
      A dictionary containing the velocity in x and y directions (vx, vy) in pixels per frame.
  """
  # Check if previous data is available
  if not prev_object_data:
    return {"vx": 0, "vy": 0}

  center_x = int(object_data[0] + object_data[2] / 2)
  center_y = int(object_data[1] + object_data[3] / 2)
  prev_center_x = int(prev_object_data[0] + prev_object_data[2] / 2)
  prev_center_y = int(prev_object_data[1] + prev_object_data[3] / 2)

  # Calculate velocity in x and y directions (pixels per frame)
  vx = (center_x - prev_center_x) / frame_rate
  vy = (center_y - prev_center_y) / frame_rate

  return {"vx": vx, "vy": vy}

In [27]:
#cap.release()

In [28]:
prev_object_data = None

In [29]:
def draw_velocity_arrow(frame, object_data, velocity, scale=10):
  """
  This function draws an arrow on a frame to represent the object's velocity.

  Args:
      frame: The frame image from the video.
      object_data: A dictionary containing object data (x, y, width, height, angle).
      velocity: A dictionary containing velocity data (vx, vy) in pixels per frame.
      scale: A factor to scale the arrow length for better visualization (optional).
  """
  if object_data and velocity:
    # Calculate arrow end point based on object center and scaled velocity
    arrow_x = int(object_data[0] + velocity["vx"] * scale)
    arrow_y = int(object_data[1] + velocity["vy"] * scale)
    end_point = (arrow_x, arrow_y)


    # Draw arrow line using cv2.arrowedLine
    cv2.arrowedLine(frame, (int(object_data[0]), int(object_data[1])), end_point,
                    (0, 0, 255), 2)  # Color: Blue

In [30]:
def draw_circle_around_ellipse(image, x, y, w, h,  circle_radius):
    # Calculate the center of the ellipse
    center_x = int(x + w / 2)
    center_y = int(y + h / 2)

    # Draw the circle around the ellipse center
    cv2.circle(image, (center_x, center_y), circle_radius, (0, 255, 0), 2)


In [31]:
circle_center = (-1, -1)
radius = 10
def draw_circle(event, x, y, flags, param):
    global circle_center, radius

    if event == cv2.EVENT_LBUTTONDOWN:
        # Update circle center
        circle_center = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        # Update radius as the distance between the current position and the center
        radius = max(1, int(((x - circle_center[0])**2 + (y - circle_center[1])**2)**0.5))
        # Draw the circle and the line on the image
        image = param.copy()
        cv2.circle(image, circle_center, radius, (0, 255, 0), 2)
        cv2.line(image, circle_center, (x, y), (0, 0, 255), 2)
        cv2.imshow("Image", image)

    elif event == cv2.EVENT_LBUTTONUP:
        # Draw the final circle on the image
        cv2.circle(param, circle_center, radius, (0, 255, 0), 2)
        # Reset circle_center and radius
        circle_center = (-1, -1)
        radius = 10
image = cv2.imread("frame_2.jpg")  # Replace "your_image_path.jpg" with the path to your image


# Create a window and bind the mouse callback function to it
cv2.namedWindow("Image")
cv2.setMouseCallback("Image", draw_circle, param=image)

while True:
    # Display the image
    cv2.imshow("Image", image)

    # Check for key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):  # Press 'q' to quit
        break

cv2.destroyAllWindows()

In [32]:
import math

def circle_position(inner_center, inner_radius, outer_center, outer_radius):
    # Calculate distance between centers
    distance = math.sqrt((outer_center[0] - inner_center[0]) ** 2 + (outer_center[1] - inner_center[1]) ** 2)
    
    # Compare distance with sum of radii
    if distance + inner_radius < outer_radius:
        return "inside"
    elif distance - inner_radius > outer_radius:
        return "outside"
    else:
        # Calculate angle between centers
        angle = math.atan2(inner_center[1] - outer_center[1], inner_center[0] - outer_center[0])
        angle = math.degrees(angle) % 360
        
        # Determine position based on angle
        if 45 < angle <= 135:
            return "top"
        elif 135 < angle <= 225:
            return "left"
        elif 225 < angle <= 315:
            return "bottom"
        else:
            return "right"

# Example usage
inner_circle_center = (2, 2)
inner_circle_radius = 1
outer_circle_center = (0, 0)
outer_circle_radius = 3

position = circle_position(inner_circle_center, inner_circle_radius, outer_circle_center, outer_circle_radius)
print("Inner circle is", position, "of outer circle.")

Inner circle is right of outer circle.


In [33]:
def circle_position(inner_center, inner_radius, outer_center, outer_radius):
    # Calculate distance between centers
    distance = math.sqrt((outer_center[0] - inner_center[0]) ** 2 + (outer_center[1] - inner_center[1]) ** 2)
    
    # Compare distance with sum of radii
    if distance + inner_radius < outer_radius:
        return "inside"
    elif distance - inner_radius > outer_radius:
        return "outside"
    else:
        # Calculate angle between centers
        angle = math.atan2(inner_center[1] - outer_center[1], inner_center[0] - outer_center[0])
        angle = math.degrees(angle) % 360
        
        # Determine position based on angle
        if 45 < angle <= 135:
            return "top"
        elif 135 < angle <= 225:
            return "left"
        elif 225 < angle <= 315:
            return "bottom"
        else:
            return "right"

In [60]:
def detect_block(detected_object_x, detected_object_y, roi):
    x, y, w, h = roi

    # Determine the coordinates of the four blocks
    block_width = w // 2
    block_height = h // 2

    # Check in which block the detected object lies
    if detected_object_x >= x and detected_object_x < x + block_width:
        if detected_object_y >= y and detected_object_y < y + block_height:
            return "top-left"
        else:
            return "bottom-left"
    elif detected_object_x >= x + block_width and detected_object_x < x + w:
        if detected_object_y >= y and detected_object_y < y + block_height:
            return "top-right"
        else:
            return "bottom-right"
    else:
        return "outside ROI"

In [121]:
cap = cv2.VideoCapture('./07.avi')
ret, frame = cap.read()
height, width, _ = frame.shape
frame = cv2.resize(frame, (width*2,height*2))
frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
roi = cv2.selectROI("selection",frame)
cv2.waitKey(0)
cv2.destroyWindow("selection")
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
height_, width_, _ = frame.shape
out_vid = cv2.VideoWriter("./test2.mp4", fourcc, frame_rate, (width_, height_))
while(cap.isOpened()):
  # Capture frame-by-frame
    ret, frame = cap.read()
    
    
    if ret == True:
        frame = cv2.resize(frame, (width*2,height*2))
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
        #selected_roi = frame[int(roi[1]):int(roi[1] + roi[3]), int(roi[0]):int(roi[0] + roi[2])]
        #selected_roi = cv2.GaussianBlur(selected_roi,(3,3),0)

        out,cnt = fitPupil(frame,circ_thresh=.1,thresh_val=45)
        #height, width, _ = out.shape
        
        

        if all(element != 0 for element in cnt):
            x,y,w,h,angle = cnt
            
            viewing_angle = detect_block(x,y,roi)
            
            # You can adjust the position as needed
            object_data = cnt

            velocity = calculate_velocity(object_data, prev_object_data, frame_rate)
            
            

           
            text_lines = [f'x: {cnt[0]:.2f}', f'y: {cnt[1]:.2f}', f'w: {cnt[2]:.2f}', f'h: {cnt[3]:.2f}',
                           f'angle: {cnt[4]:.2f}',f'vx: {velocity["vx"]:.2f}',
                           f'vy: {velocity["vy"]:.2f}', f"viewing angle: {viewing_angle}"
                           ]
            
            if velocity:
              #pass
              draw_velocity_arrow(out,object_data,velocity)

            #resized_out = cv2.resize(out, (width,height))
            
            center_x = int(x/ 2)
            center_y = int(y / 2)

            

            # Draw circle in the middle
            #cv2.circle(out, (int(x)+100, int(y)+100), 180, (0, 255, 0), 1)
            #draw_circle_around_ellipse(resized_out, center_x , center_y, cnt[2], cnt[3], 100)
            draw_text(out, text_lines, (10, 30))
            x, y, w, h = roi
            cv2.rectangle(out, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.imshow("Frame",out)
            #print(x,y,w,h,angle)
           
            
            prev_object_data = object_data
            out_vid.write(out)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                out_vid.release()
                cap.release()
                cv2.destroyAllWindows()
                break
        else:
          cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
          cv2.imshow("Frame",frame)
          out_vid.write(frame)
          if cv2.waitKey(3) & 0xFF == ord('q'):
              
              out_vid.release()
              cap.release()
              cv2.destroyAllWindows()
              break 
    # Break the loop
    
    else: break
cap.release()
out_vid.release()
cv2.destroyAllWindows()

In [11]:
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [135]:
import numpy as np

def get_gaze_direction(x, y, w, h, angle, image_width, image_height):
    # Convert angle to radians
    angle_rad = np.deg2rad(angle)
    
    # Calculate the center of the ellipse
    center_x = x + w / 2
    center_y = y + h / 2
    
    # Calculate the direction vector based on angle
    direction_vector = np.array([np.cos(angle_rad), np.sin(angle_rad)])
    
    # Normalize the direction vector
    direction_vector /= np.linalg.norm(direction_vector)
    
    # Calculate the dot product of direction vector and horizontal/vertical axis
    horizontal_dot = np.dot(direction_vector, [1, 0])  # Horizontal axis
    vertical_dot = np.dot(direction_vector, [0, 1])    # Vertical axis
    
    # Determine the direction based on dot products
    if abs(horizontal_dot) > abs(vertical_dot):
        if horizontal_dot > 0:
            return "Right"
        else:
            return "Left"
    else:
        if vertical_dot > 0:
            return "Down"
        else:
            return "Up"

# Example usage
x = 100  # Example ellipse x-coordinate
y = 150  # Example ellipse y-coordinate
w = 50   # Example ellipse width
h = 30   # Example ellipse height
angle = 30  # Example ellipse angle
image_width = 640  # Example image width
image_height = 480  # Example image height

direction = get_gaze_direction(x, y, w, h, angle, image_width, image_height)
print("Gaze direction:", direction)

Gaze direction: Right


In [19]:
def determine_block(image, roi, object_x, object_y):
    # Extract ROI coordinates
    x, y, w, h = roi
    roi_center_x = x + w // 2
    roi_center_y = y + h // 2

    # Define quadrants
    quadrants = {
        "top-left": (x, y, roi_center_x, roi_center_y),
        "top-right": (roi_center_x, y, x + w, roi_center_y),
        "bottom-left": (x, roi_center_y, roi_center_x, y + h),
        "bottom-right": (roi_center_x, roi_center_y, x + w, y + h)
    }

    # Check if the object is inside the ROI
    if x <= object_x <= x + w and y <= object_y <= y + h:
        # Determine which quadrant the object belongs to
        for quadrant, (start_x, start_y, end_x, end_y) in quadrants.items():
            if start_x <= object_x <= end_x and start_y <= object_y <= end_y:
                return quadrant
    
    # If the object is not inside the ROI
    return "Object not inside ROI"


In [18]:
cv2.destroyAllWindows()
im = cv2.imread("frame_2.jpg")
r = cv2.selectROI(im)

x, y, w, h = r
roi_center_x = x + w // 2
roi_center_y = y + h // 2

# Define quadrants
quadrants = {
    "top-left": (x, y, roi_center_x, roi_center_y),
    "top-right": (roi_center_x, y, x + w, roi_center_y),
    "bottom-left": (x, roi_center_y, roi_center_x, y + h),
    "bottom-right": (roi_center_x, roi_center_y, x + w, y + h)
}

# Draw rectangles for each quadrant
for quadrant, (start_x, start_y, end_x, end_y) in quadrants.items():
    cv2.rectangle(im, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
    cv2.putText(im, quadrant, (start_x + 10, start_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)


cv2.imshow('ROI with Quadrants', im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [40]:
cv2.destroyAllWindows()
image = cv2.imread("frame_178.jpg")
out,cnt = fitPupil(image,circ_thresh=.5,thresh_val=45)
print(cnt)
roi = cv2.selectROI(image)
object_x = 100  # Example x coordinate of the object
object_y = 150  # Example y coordinate of the object
block = determine_block(image, roi, object_x, object_y)

print("Object is in", block)

[0.0, 0.0, 0.0, 0.0, 0.0]
Object is in Object not inside ROI
